# Yolo Components

In [1]:
import os
os.getcwd()
# os.chdir('D:\Python Projects\EVA\TheCapStone')

'd:\\Python Projects\\EVA\\15_TheCapStone'

In [2]:
#Load the weights from pre-trained model
os.chdir("./models_all/YoloV3/")

from models import *
from utils.parse_config import *

def get_yolo():

    path = './cfg/yolov3-custom.cfg'
    yolo_model = Darknet(path).to("cuda")

    sc_yolokeys = list(yolo_model.state_dict().keys())
    model_path = r".\pretrained_model\best273.pt"
    ptmodel = torch.load(model_path)['model']
    pt_yolokeys = (list(ptmodel.keys())[330:])
    # Updating the Yolo model with Pre-trained weights
    for i, keys in enumerate(sc_yolokeys):
        yolo_model.state_dict()[keys] = ptmodel[pt_yolokeys[i]]
    # Freeze the layers for training
    for param in yolo_model.parameters():
        param.requires_grad = False
    return yolo_model

In [3]:
yolo_model = get_yolo()

Now adding layers
Model Summary: 60 layers, 1.5916e+07 parameters, 1.5916e+07 gradients


# Midas Components

In [7]:
# Loading the pretrained encoder of Midas
os.chdir(os.path.dirname(os.getcwd()))
os.chdir(r'.\midas')
from midas.midas_net import *
def get_midas():
    model_path = r"D:\Python Projects\EVA\15_TheCapStone\models_all\midas\model-f6b98070.pt"
    midas_model = MidasNet(model_path, non_negative=True).to("cuda")
    # Freezing layers for updates
    for param in midas_model.parameters():
        param.requires_grad_(False)
    return midas_model


In [8]:
midas_model = get_midas()

Loading weights:  D:\Python Projects\EVA\15_TheCapStone\models_all\midas\model-f6b98070.pt
Using cache found in C:\Users\BlueFlames/.cache\torch\hub\facebookresearch_WSL-Images_master
Path D:\Python Projects\EVA\15_TheCapStone\models_all\midas\model-f6b98070.pt


# Ensemble Model

In [18]:
# Emsemble model
class MyEnsemble(nn.Module):
    def __init__(self, midas, yolo_decoder, train_yolo=False, train_rcnn=False, train_all=True):
        super(MyEnsemble, self).__init__()
        self.midas = midas
        self.yolo_decoder = yolo_decoder
        self.train_yolo = train_yolo
        self.train_rcnn = train_rcnn
        self.train_all = train_all
        # Remove last linear layer
        # self.modelA.fc = nn.Identity()
        # self.modelB.fc = nn.Identity()
        
        # Create a 3 Layer buffer between Midas Encoder and Yolo Decoder
        # Everything would be freezed but this
        
        self.yolo_buffer = nn.Sequential(
            nn.Conv2d(2048, 2048, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(2048),
            nn.ReLU(True),
            nn.Conv2d(2048, 2048, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(2048),
            nn.ReLU(True),
            nn.Conv2d(2048, 2048, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(2048),
            nn.ReLU(True)
        )

        
    def forward(self, x):

        layer_1 = self.midas.pretrained.layer1(x.clone())
        layer_2 = self.midas.pretrained.layer2(layer_1)
        layer_3 = self.midas.pretrained.layer3(layer_2)
        layer_4 = self.midas.pretrained.layer4(layer_3)

        # if self.train_all:
        layer_1_rn = self.midas.scratch.layer1_rn(layer_1)
        layer_2_rn = self.midas.scratch.layer2_rn(layer_2)
        layer_3_rn = self.midas.scratch.layer3_rn(layer_3)
        layer_4_rn = self.midas.scratch.layer4_rn(layer_4)

        path_4 = self.midas.scratch.refinenet4(layer_4_rn)
        path_3 = self.midas.scratch.refinenet3(path_4, layer_3_rn)
        path_2 = self.midas.scratch.refinenet2(path_3, layer_2_rn)
        path_1 = self.midas.scratch.refinenet1(path_2, layer_1_rn)

        out = self.midas.scratch.output_conv(path_1)

        # Now adding a buffer and Yolo Layers
        yolo_buffer = self.yolo_buffer(layer_4)
        yolo_output = self.yolo_decoder(yolo_buffer)         

        # return torch.squeeze(out, dim=1), yolo_output
        
        # if self.train_yolo:
        #     yolo_buffer = self.yolo_buffer(layer_4)
        #     yolo_output = self.yolo_decoder(yolo_buffer)         

        return torch.squeeze(out, dim=1)

In [19]:
# Create ensemble model
model = MyEnsemble(midas_model, yolo_model).to("cuda")

In [8]:
data = torch.rand((1,3,416,416)).to("cuda")

In [11]:
from torchsummary import summary
summary(model, input_size=(3,416,416))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 208, 208]           9,408
       BatchNorm2d-2         [-1, 64, 208, 208]             128
              ReLU-3         [-1, 64, 208, 208]               0
         MaxPool2d-4         [-1, 64, 104, 104]               0
            Conv2d-5        [-1, 256, 104, 104]          16,384
       BatchNorm2d-6        [-1, 256, 104, 104]             512
              ReLU-7        [-1, 256, 104, 104]               0
            Conv2d-8        [-1, 256, 104, 104]          18,432
       BatchNorm2d-9        [-1, 256, 104, 104]             512
             ReLU-10        [-1, 256, 104, 104]               0
           Conv2d-11        [-1, 256, 104, 104]          65,536
      BatchNorm2d-12        [-1, 256, 104, 104]             512
           Conv2d-13        [-1, 256, 104, 104]          16,384
      BatchNorm2d-14        [-1, 256, 1

In [1]:
os.getcwd()
os.chdir(r"./models_all/midas")

In [22]:
import os
import glob
import torch
from models_all.midas.utils import *
import cv2

from torchvision.transforms import Compose
from midas.midas_net import MidasNet
from midas.transforms import Resize, NormalizeImage, PrepareForNet


def run(input_path, output_path, model):
    """Run MonoDepthNN to compute depth maps.

    Args:
        input_path (str): path to input folder
        output_path (str): path to output folder
        model_path (str): path to saved model
    """
    print("initialize")

    # select device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("device: %s" % device)

    # load network

    transform = Compose(
        [
            Resize(
                384,
                384,
                resize_target=None,
                keep_aspect_ratio=True,
                ensure_multiple_of=32,
                resize_method="upper_bound",
                image_interpolation_method=cv2.INTER_CUBIC,
            ),
            NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            PrepareForNet(),
        ]
    )

    model.to(device)
    model.eval()

    # get input
    img_names = glob.glob(os.path.join(input_path, "*"))
    num_images = len(img_names)

    # create output folder
    os.makedirs(output_path, exist_ok=True)

    print("start processing")

    for ind, img_name in enumerate(img_names):

        print("  processing {} ({}/{})".format(img_name, ind + 1, num_images))

        # input

        img = read_image(img_name)
        img_input = transform({"image": img})["image"]

        # compute
        with torch.no_grad():
            sample = torch.from_numpy(img_input).to(device).unsqueeze(0)
            prediction = model.forward(sample)
            prediction = (
                torch.nn.functional.interpolate(
                    prediction.unsqueeze(1),
                    size=img.shape[:2],
                    mode="bicubic",
                    align_corners=False,
                )
                .squeeze()
                .cpu()
                .numpy()
            )

        # output
        filename = os.path.join(
            output_path, os.path.splitext(os.path.basename(img_name))[0]
        )
        write_depth(filename, prediction, bits=2)

    print("finished")


In [23]:

    # set paths
    INPUT_PATH = r"D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images"
    OUTPUT_PATH = r"D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\output"
    # MODEL_PATH = "model.pt"

    # set torch options
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

    # compute depth maps
    run(INPUT_PATH, OUTPUT_PATH, model)

initialize
device: cuda
start processing
  processing D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images\z053 .jpg (1/12)
  processing D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images\z054 .jpg (2/12)
  processing D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images\z055 .jpg (3/12)
  processing D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images\z056 .jpg (4/12)
  processing D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images\z057 .jpg (5/12)
  processing D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images\z058 .jpg (6/12)
  processing D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images\z059 .jpg (7/12)
  processing D:\Python Projects\EVA\15_TheCapStone\models_all\YoloV3\data\customdata\data\images\z060 .jpg (8/12)
  processing D:\Python Projects\EVA\15_TheCapSt

In [3]:
import torch
pre_trained_model = torch.load(r"D:\Python Projects\EVA\15_TheCapStone\models_all\midas\model-f6b98070.pt")

In [5]:
print(pre_trained_model.keys())

odict_keys(['pretrained.layer1.0.weight', 'pretrained.layer1.1.weight', 'pretrained.layer1.1.bias', 'pretrained.layer1.1.running_mean', 'pretrained.layer1.1.running_var', 'pretrained.layer1.1.num_batches_tracked', 'pretrained.layer1.4.0.conv1.weight', 'pretrained.layer1.4.0.bn1.weight', 'pretrained.layer1.4.0.bn1.bias', 'pretrained.layer1.4.0.bn1.running_mean', 'pretrained.layer1.4.0.bn1.running_var', 'pretrained.layer1.4.0.bn1.num_batches_tracked', 'pretrained.layer1.4.0.conv2.weight', 'pretrained.layer1.4.0.bn2.weight', 'pretrained.layer1.4.0.bn2.bias', 'pretrained.layer1.4.0.bn2.running_mean', 'pretrained.layer1.4.0.bn2.running_var', 'pretrained.layer1.4.0.bn2.num_batches_tracked', 'pretrained.layer1.4.0.conv3.weight', 'pretrained.layer1.4.0.bn3.weight', 'pretrained.layer1.4.0.bn3.bias', 'pretrained.layer1.4.0.bn3.running_mean', 'pretrained.layer1.4.0.bn3.running_var', 'pretrained.layer1.4.0.bn3.num_batches_tracked', 'pretrained.layer1.4.0.downsample.0.weight', 'pretrained.layer1.4.

In [ ]:
import torchsummary
torchsummary(pre_trained_model, 

In [10]:
import os
os.chdir('d:\\Python Projects\\EVA\\15_TheCapStone\\models_all\\YoloV3')

In [11]:
import argparse

import torch.distributed as dist
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from test_model import *  # import test.py to get mAP after each epoch
from models import *
from utils.datasets import *
from utils.utils import *


results_file = 'results.txt'

hyp = {'giou': 3.54,  # giou loss gain
       'cls': 37.4,  # cls loss gain
       'cls_pw': 1.0,  # cls BCELoss positive_weight
       'obj': 64.3,  # obj loss gain (*=img_size/320 if img_size != 320)
       'obj_pw': 1.0,  # obj BCELoss positive_weight
       'iou_t': 0.225,  # iou training threshold
       'lr0': 0.01,  # initial learning rate (SGD=5E-3, Adam=5E-4)
       'lrf': 0.0005,  # final learning rate (with cos scheduler)
       'momentum': 0.937,  # SGD momentum
       'weight_decay': 0.000484,  # optimizer weight decay
       'fl_gamma': 0.0,  # focal loss gamma (efficientDet default is gamma=1.5)
       'hsv_h': 0.0138,  # image HSV-Hue augmentation (fraction)
       'hsv_s': 0.678,  # image HSV-Saturation augmentation (fraction)
       'hsv_v': 0.36,  # image HSV-Value augmentation (fraction)
       'degrees': 1.98 * 0,  # image rotation (+/- deg)
       'translate': 0.05 * 0,  # image translation (+/- fraction)
       'scale': 0.05 * 0,  # image scale (+/- gain)
       'shear': 0.641 * 0}  # image shear (+/- deg)

# Overwrite hyp with hyp*.txt (optional)
f = glob.glob('hyp*.txt')
if f:
    print('Using %s' % f[0])
    for k, v in zip(hyp.keys(), np.loadtxt(f[0])):
        hyp[k] = v


def train(model):
    # Value setting for hyperparameters
    device = "cuda"
    data = r'.\data\customdata\custom.data' # Path in which train images are located # path = './cfg/yolov3-custom.cfg'
    epochs = 10  # 500200 batches at bs 64, 117263 images = 273 epochs
    batch_size = 1
    accumulate = 4  # effective bs = batch_size * accumulate = 16 * 4 = 64
    img_size = 512
    imgsz_test = 512
   
    # Configure run
    init_seeds() # Set seeds
    data_dict = parse_data_cfg(data) #parse the config
    print(data_dict['valid'])
    train_path = r".\data\customdata\custom.txt"
    test_path = r".\data\customdata\custom.txt"
    custom_names = r".\data\customdata\custom.names"
    nc = 4  # number of classes
    hyp['cls'] *= nc / 80  # update coco-tuned hyp['cls'] to current dataset

    cfg = './cfg/yolov3-custom.cfg'

    # Initialize model
    # model = model

    pg0, pg1, pg2 = [], [], []  # optimizer parameter groups
    for k, v in dict(model.named_parameters()).items():
        if '.bias' in k:
            pg2 += [v]  # biases
        elif 'Conv2d.weight' in k:
            pg1 += [v]  # apply weight_decay
        else:
            pg0 += [v]  # all else
    
    # Initializing the optimizer
    optimizer = optim.SGD(pg0, lr=hyp['lr0'], momentum=hyp['momentum'], nesterov=True)
    optimizer.add_param_group({'params': pg1, 'weight_decay': hyp['weight_decay']})  # add pg1 with weight_decay
    optimizer.add_param_group({'params': pg2})  # add pg2 (biases)
    del pg0, pg1, pg2

    start_epoch = 0
    best_fitness = 0.0

    lf = lambda x: (((1 + math.cos(
        x * math.pi / epochs)) / 2) ** 1.0) * 0.95 + 0.05  # cosine https://arxiv.org/pdf/1812.01187.pdf
    
    # train_path = 
    # LR scheduler
#################################################################################################
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf, last_epoch=start_epoch - 1)

    # Reading custom dataset and applying custom augmentation
    dataset = LoadImagesAndLabels(train_path, img_size, batch_size,
                                  augment=False,
                                  hyp=hyp,  # augmentation hyperparameters
                                  )
############################################################################################
    # Dataloader - Loading the dataset
    batch_size = min(batch_size, len(dataset))
    nw = 1 #min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
    dataloader = torch.utils.data.DataLoader(dataset,
                                             batch_size=batch_size,
                                             num_workers=nw,
                                             shuffle=True,  # Shuffle=True unless rectangular training isused
                                             pin_memory=True,
                                             collate_fn=dataset.collate_fn)

    # Testloader
    testloader = torch.utils.data.DataLoader(LoadImagesAndLabels(test_path, imgsz_test, batch_size,hyp=hyp),
                                             batch_size=batch_size,
                                             num_workers=nw,
                                             pin_memory=True,
                                             collate_fn=dataset.collate_fn)


    # Model Training parameters
    model.nc = 4  # attach number of classes to model
    model.hyp = hyp  # attach hyperparameters to model
    model.gr = 1.0  # giou loss ratio (obj_loss = 1.0 or giou)
    # model.class_weights = labels_to_class_weights(dataset.labels, nc).to(device)  # attach class weights konse wale

    # Model EMA - Is this necessary? EMA - Moving averages weight setting
    ema = torch_utils.ModelEMA(model)

    # Start training
    nb = len(dataloader)  # number of batches
    n_burn = max(3 * nb, 500)  # burn-in iterations, max(3 epochs, 500 iterations)
    maps = np.zeros(nc)  # mAP per class
    results = (0, 0, 0, 0, 0, 0, 0)  # 'P', 'R', 'mAP', 'F1', 'val GIoU', 'val Objectness', 'val Classification'
    t0 = time.time()
    # print('Image sizes %g - %g train, %g test' % (imgsz_min, imgsz_max, ))
    
    print('Using %g dataloader workers' % nw)
    print('Starting training for %g epochs...' % epochs)

    # Training loop
    for epoch in range(start_epoch, epochs):  # epoch ------------------------------------------------------------------
        model.train()
        # loss
        mloss = torch.zeros(4).to(device)  # mean losses
        print(('\n' + '%10s' * 8) % ('Epoch', 'gpu_mem', 'GIoU', 'obj', 'cls', 'total', 'targets', 'img_size'))
        pbar = tqdm(enumerate(dataloader), total=nb)  # progress bar

        for i, (imgs, targets, paths, _) in pbar:  # batch -------------------------------------------------------------
            ni = i + nb * epoch  # number integrated batches (since train start)
            imgs = imgs.to(device).float() / 255.0  # uint8 to float32, 0 - 255 to 0.0 - 1.0
            targets = targets.to(device)

            # Burn-in
            if ni <= n_burn * 2:
                model.gr = np.interp(ni, [0, n_burn * 2], [0.0, 1.0])  # giou loss ratio (obj_loss = 1.0 or giou)
                if ni == n_burn:  # burnin complete
                    print_model_biases(model)

                for j, x in enumerate(optimizer.param_groups):
                    # bias lr falls from 0.1 to lr0, all other lrs rise from 0.0 to lr0
                    x['lr'] = np.interp(ni, [0, n_burn], [0.1 if j == 2 else 0.0, x['initial_lr'] * lf(epoch)])
                    if 'momentum' in x:
                        x['momentum'] = np.interp(ni, [0, n_burn], [0.9, hyp['momentum']])

            # Run model
            pred = model(imgs)

            # Compute loss
            loss, loss_items = compute_loss(pred, targets, model)
            if not torch.isfinite(loss):
                print('WARNING: non-finite loss, ending training ', loss_items)
                return results

            # Scale loss by nominal batch_size of 64
            loss *= batch_size / 64


            loss.backward()

            # Optimize accumulated gradient
            if ni % accumulate == 0:
                optimizer.step()
                optimizer.zero_grad()
                ema.update(model)

            # Print batch results
            mloss = (mloss * i + loss_items) / (i + 1)  # update mean losses
            mem = '%.3gG' % (torch.cuda.memory_cached() / 1E9 if torch.cuda.is_available() else 0)  # (GB)
            s = ('%10s' * 2 + '%10.3g' * 6) % ('%g/%g' % (epoch, epochs - 1), mem, *mloss, len(targets), img_size)
            pbar.set_description(s)

            # Plot images with bounding boxes
            # print('@@@@@ ni values are',ni)
            if ni < 1:
                f = 'train_batch%g.png' % i  # filename
                plot_images(imgs=imgs, targets=targets, paths=paths, fname=f)      

            # end batch ------------------------------------------------------------------------------------------------

        # Update scheduler
        scheduler.step()

        # Process epoch results
        ema.update_attr(model)
        final_epoch = epoch + 1 == epochs
        # if final_epoch:  # Calculate mAP
        is_coco = any([x in data for x in ['coco.data', 'coco2014.data', 'coco2017.data']]) and model.nc == 80
        results, maps = test_model(cfg,
                            data,
                            batch_size=batch_size,
                            img_size=imgsz_test,
                            model=ema.ema,
                            save_json=final_epoch and is_coco,
                            single_cls=False,
                            dataloader=testloader)

        # Write epoch results
        with open(results_file, 'a') as f:
            f.write(s + '%10.3g' * 7 % results + '\n')  # P, R, mAP, F1, test_losses=(GIoU, obj, cls)
               # Update best mAP
        fi = fitness(np.array(results).reshape(1, -1))  # fitness_i = weighted combination of [P, R, mAP, F1]
        if fi > best_fitness:
            best_fitness = fi
        # end epoch ----------------------------------------------------------------------------------------------------


In [12]:
import argparse
import json

from torch.utils.data import DataLoader

from models import *
from utils.datasets import *
from utils.utils import *


def test_model(cfg,
         data,
         weights=None,
         batch_size=16,
         img_size=416,
         conf_thres=0.001,
         iou_thres=0.6,  # for nms
         save_json=False,
         single_cls=False,
         augment=False,
         model=None,
         dataloader=None):
    # Initialize/load model and set device
  

    device = next(model.parameters()).device
    verbose = False

    # Configure run
    data = parse_data_cfg(data)
    nc = 1 if single_cls else int(data['classes'])  # number of classes
    path = data['valid']  # path to test images


    names = ['hardhat', 'vest', 'mask', 'boots']  # class names, yaha panga hai
    iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
    iouv = iouv[0].view(1)  # comment for mAP@0.5:0.95
    niou = iouv.numel()

    seen = 0
    model.eval()

    _ = model(torch.zeros((1, 3, img_size, img_size), device=device)) if device.type != 'cpu' else None  # run once
    coco91class = coco80_to_coco91_class()
    s = ('%20s' + '%10s' * 6) % ('Class', 'Images', 'Targets', 'P', 'R', 'mAP@0.5', 'F1')
    p, r, f1, mp, mr, map, mf1, t0, t1 = 0., 0., 0., 0., 0., 0., 0., 0., 0.
    loss = torch.zeros(3, device=device)
    jdict, stats, ap, ap_class = [], [], [], []
    for batch_i, (imgs, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
        imgs = imgs.to(device).float() / 255.0  # uint8 to float32, 0 - 255 to 0.0 - 1.0
        targets = targets.to(device)
        nb, _, height, width = imgs.shape  # batch size, channels, height, width
        whwh = torch.Tensor([width, height, width, height]).to(device)

        f = 'test_batch%g.png' % batch_i  # filename

        plot_images(imgs=imgs, targets=targets, paths=paths, fname=f)

        # Plot images with bounding boxes
        f = 'test_batch%g.png' % batch_i  # filename
        if batch_i < 1 and not os.path.exists(f):
            plot_images(imgs=imgs, targets=targets, paths=paths, fname=f)

        # Disable gradients
        with torch.no_grad():
            # Run model
            t = torch_utils.time_synchronized()
            inf_out, train_out = model(imgs)  # inference and training outputs
            t0 += torch_utils.time_synchronized() - t

            # Compute loss
            if hasattr(model, 'hyp'):  # if model has loss hyperparameters
                loss += compute_loss(train_out, targets, model)[1][:3]  # GIoU, obj, cls

            # Run NMS
            t = torch_utils.time_synchronized()
            output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)  # nms
            t1 += torch_utils.time_synchronized() - t

        # Statistics per image
        for si, pred in enumerate(output):
            labels = targets[targets[:, 0] == si, 1:]
            nl = len(labels)
            tcls = labels[:, 0].tolist() if nl else []  # target class
            seen += 1

            if pred is None:
                if nl:
                    stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
                continue

            # Clip boxes to image bounds
            clip_coords(pred, (height, width))

            # Append to pycocotools JSON dictionary
            if save_json:
                # [{"image_id": 42, "category_id": 18, "bbox": [258.15, 41.29, 348.26, 243.78], "score": 0.236}, ...
                image_id = int(Path(paths[si]).stem.split('_')[-1])
                box = pred[:, :4].clone()  # xyxy
                scale_coords(imgs[si].shape[1:], box, shapes[si][0], shapes[si][1])  # to original shape
                box = xyxy2xywh(box)  # xywh
                box[:, :2] -= box[:, 2:] / 2  # xy center to top-left corner
                for p, b in zip(pred.tolist(), box.tolist()):
                    jdict.append({'image_id': image_id,
                                  'category_id': coco91class[int(p[5])],
                                  'bbox': [round(x, 3) for x in b],
                                  'score': round(p[4], 5)})

            # Assign all predictions as incorrect
            correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool, device=device)
            if nl:
                detected = []  # target indices
                tcls_tensor = labels[:, 0]

                # target boxes
                tbox = xywh2xyxy(labels[:, 1:5]) * whwh

                # Per target class
                for cls in torch.unique(tcls_tensor):
                    ti = (cls == tcls_tensor).nonzero().view(-1)  # prediction indices
                    pi = (cls == pred[:, 5]).nonzero().view(-1)  # target indices

                    # Search for detections
                    if pi.shape[0]:
                        # Prediction to target ious
                        ious, i = box_iou(pred[pi, :4], tbox[ti]).max(1)  # best ious, indices

                        # Append detections
                        for j in (ious > iouv[0]).nonzero():
                            d = ti[i[j]]  # detected target
                            if d not in detected:
                                detected.append(d)
                                correct[pi[j]] = ious[j] > iouv  # iou_thres is 1xn
                                if len(detected) == nl:  # all targets already located in image
                                    break

            # Append statistics (correct, conf, pcls, tcls)
            stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))

    # Compute statistics
    stats = [np.concatenate(x, 0) for x in zip(*stats)]  # to numpy
    if len(stats):
        p, r, ap, f1, ap_class = ap_per_class(*stats)
        if niou > 1:
            p, r, ap, f1 = p[:, 0], r[:, 0], ap.mean(1), ap[:, 0]  # [P, R, AP@0.5:0.95, AP@0.5]
        mp, mr, map, mf1 = p.mean(), r.mean(), ap.mean(), f1.mean()
        nt = np.bincount(stats[3].astype(np.int64), minlength=nc)  # number of targets per class
    else:
        nt = torch.zeros(1)

    # Print results
    pf = '%20s' + '%10.3g' * 6  # print format
    print(pf % ('all', seen, nt.sum(), mp, mr, map, mf1))

    # Print results per class
    if verbose and nc > 1 and len(stats):
        for i, c in enumerate(ap_class):
            print(pf % (names[c], seen, nt[c], p[i], r[i], ap[i], f1[i]))

    # Print speeds
    if verbose or save_json:
        t = tuple(x / seen * 1E3 for x in (t0, t1, t0 + t1)) + (img_size, img_size, batch_size)  # tuple
        print('Speed: %.1f/%.1f/%.1f ms inference/NMS/total per %gx%g image at batch-size %g' % t)


    maps = np.zeros(nc) + map
    for i, c in enumerate(ap_class):
        maps[c] = ap[i]
    return (mp, mr, map, mf1, *(loss.cpu() / len(dataloader)).tolist()), maps

In [13]:
train(model)

Caching labels (11 found, 0 missing, 0 empty, 0 duplicate, for 11 images): 100%|██████████| 11/11 [00:00<00:00, 1102.89it/s]
Caching labels (11 found, 0 missing, 0 empty, 0 duplicate, for 11 images): 100%|██████████| 11/11 [00:00<00:00, 2221.13it/s]./customdata/data/test.txt
Using 1 dataloader workers
Starting training for 10 epochs...

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size

       0/9     3.34G      8.71      9.44      3.16      21.3        13       512:   0%|          | 0/11 [00:02<?, ?it/s]@@@@@ ni values are 0
       0/9     3.34G      6.81       8.4      2.76        18         6       512:  18%|█▊        | 2/11 [00:02<00:17,  1.93s/it]@@@@@ ni values are 1
       0/9     3.34G      7.42      7.89      2.92      18.2         9       512:  27%|██▋       | 3/11 [00:03<00:11,  1.48s/it]@@@@@ ni values are 2
       0/9     3.34G      7.61      7.72      3.03      18.4         9       512:  36%|███▋      | 4/11 [00:03<00:08,  1.16s/it]@@@@@ ni 

UnboundLocalError: local variable 'f' referenced before assignment

In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [15]:
model.yolo_decoder.module_list[1]

Sequential(
  (Conv2d): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (BatchNorm2d): BatchNorm2d(1024, eps=0.0001, momentum=0.03, affine=True, track_running_stats=True)
  (activation): LeakyReLU(negative_slope=0.1, inplace=True)
)

# Planercnn model

In [16]:
%cd ".\models_all\planercnn"

[WinError 3] The system cannot find the path specified: '.\\models_all\\planercnn'
d:\Python Projects\EVA\15_TheCapStone


In [17]:
# from planercnn.train_planercnn import *

import torch
from torch import optim
from torch.utils.data import DataLoader

import os
from tqdm import tqdm
import numpy as np
import cv2
import sys

from models.model import *
from models.refinement_net import *
from models.modules import *
from datasets.plane_stereo_dataset import *

from utils import *
from visualize_utils import *
from evaluate_utils import *
from options import parse_args
from config import PlaneConfig

from argparse import Namespace

def rand_func(**kwargs):
    # print('Received kwargs', kwargs)
    args = Namespace(LR=1e-05,
        MaskRCNNPath='../mask_rcnn_coco.pth',
        anchorFolder='anchors/',
        anchorType='normal',
        batchSize=16, blocks='', checkpoint_dir='checkpoint/planercnn_normal_warping_refine',
        considerPartial=False, convType='2', cornerLocationNoise=0.0, cornerPositiveWeight=0,
        correctionType='one', customDataFolder='test/custom', dataFolder='../../Data/ScanNet/',
        dataset='', debug=False, distanceThreshold2D=20, distanceThreshold3D=0.2, frameGap=20,
        gpu=1, heatmapThreshold=0.5, height=512, keyname='planercnn_normal_warping_refine',
        locationNoise=0.0, losses='', maskHeight=56, maskWeight=1, maskWidth=56, methods='b',
        minNumPointRatio=0.05, modelType='', numAnchorPlanes=0, numEpochs=1000, numInputChannels=4,
        numNodes=10, numPositiveExamples=200, numTestingImages=100, numTrainingImages=1000,
        numViews=0, occlusionNoise=0, outputDim=256, planeAreaThreshold=500, planeWidthThreshold=10,
        positiveWeight=0.33, predictAdjacency=False, restore=2, savePoints=False, scaleMode='variant',
        startEpoch=-1, suffix='warping_refine', task='train', test_dir='test/planercnn_normal_warping_refine',
        testingDataset='', testingIndex=-1, trainingMode='all', visualizeMode='', warpingWeight=0.1, width=640
        )

    config = PlaneConfig(args)
    model = MaskRCNN(config)
    model.load_state_dict(torch.load('/content/planercnn/checkpoint/planercnn_normal_warping_refine/checkpoint.pth'))
    return model


ModuleNotFoundError: No module named 'models.model'; 'models' is not a package